<a href="https://colab.research.google.com/github/darknet-doll/colab_error_logger/blob/main/Sample_Notebook_Impact_Logger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages

In [19]:
# Import from package from error log package from github.
# Ensure error_logs.db is not in current runtime.
!pip install git+https://github.com/darknet-doll/colab_error_logger.git --upgrade
from colab_error_logger import start_logger

# Import python packagess.
import sqlite3, pandas as pd

# Airtable dependencies
!pip install pyairtable pandas
# Airtable packagess
from pyairtable import Table

# Credentials
from google.colab import userdata
AIRTABLE_TOKEN = userdata.get("AIRTABLE_TOKEN")
BASE_ID = userdata.get("AIRTABLE_BASE_ID")
TABLE_NAME = "Errors"

# Log Error

In [2]:
# Start logging errors by creating session_name field.
logger = start_logger("test_session")

In [6]:
# Testing errors.

# 1 / 0 # trigger an error.

a = error_syntax # undefined variable cannot be assigned.

NameError: name 'error_syntax' is not defined

In [7]:
# Reviewing errors logged in dataframe for this session.
session_log_df = pd.read_sql_query("SELECT * FROM errors", sqlite3.connect("error_logs.db"))
session_log_df

,id,session_name,error_type,date
0,1,test_session,ZeroDivisionError,2025-10-27
1,2,test_session,NameError,2025-10-27


# Airtable Connection

In [8]:
# Connect to table
table = Table(AIRTABLE_TOKEN, BASE_ID, TABLE_NAME)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.1/101.1 kB 3.2 MB/s eta 0:00:00


/tmp/ipython-input-2177727919.py:12: DeprecationWarning: Passing API keys or base IDs to pyairtable.Table is deprecated; use Api.table() or Base.table() instead. See https://pyairtable.rtfd.org/en/latest/migrations.html for details.
  table = Table(AIRTABLE_TOKEN, BASE_ID, TABLE_NAME)


In [16]:
# Read records from table.
records = table.all()

# Convert to DataFrame
airtable_error_df = pd.DataFrame([r["fields"] for r in records])
airtable_error_df.sort_values('count_id').head(10) # Sort table & view top 10.

,count_id,date,session_name,error_type
19,1,2025-09-17,NaN,NaN
20,2,2025-09-21,NaN,NaN
7,3,2025-09-25,NaN,NaN
2,4,NaN,NaN,NaN
8,5,2025-10-27,test_session,ZeroDivisionError
25,6,2025-10-27,test_session,NameError
21,7,2025-10-27,test_session,ZeroDivisionError
11,8,2025-10-27,test_session,InvalidJSONError
1,9,2025-10-27,test_session,HTTPError
22,10,2025-10-27,test_session,HTTPError


In [12]:
# Write to airtable_error_df by iterating through each row from session_log_df
# (excluding id column).
for _, row in session_log_df.iloc[:, 1:].iterrows():
    table.create(row.to_dict())